In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(777)

In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [4]:
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335405, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418386 ],
       [ 0.32009882,  0.52678406, -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [5]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36615504e-02 7.90546089e-03 9.78433013e-01]
 [3.93036343e-02 1.70599222e-02 9.43636417e-01]
 [3.80559057e-01 1.67907596e-01 4.51533347e-01]
 [3.23789984e-01 5.91979027e-02 6.17012143e-01]
 [3.63842742e-06 6.22515302e-08 9.99996305e-01]
 [2.63175126e-02 1.07621495e-02 9.62920368e-01]
 [1.56979695e-05 4.23308279e-07 9.99983907e-01]
 [2.95080940e-06 3.82725744e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [6]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9301591  0.06206412 0.00777684]], shape=(1, 3), dtype=float32)


In [7]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.0776258, shape=(), dtype=float32)


In [8]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [9]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.06942126, -0.6509208 ,  0.58154106],
       [-1.5220438 , -1.2148396 ,  2.7368927 ],
       [-1.2470366 , -1.7610317 ,  3.008089  ],
       [-1.2010828 , -1.8658406 ,  3.0669537 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.15204325, -0.3421458 ,  0.49418908], dtype=float32)>]


In [10]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 2.849350
Loss at epoch 100: 0.683039
Loss at epoch 200: 0.614806
Loss at epoch 300: 0.559298
Loss at epoch 400: 0.508707
Loss at epoch 500: 0.461214
Loss at epoch 600: 0.416610
Loss at epoch 700: 0.370053
Loss at epoch 800: 0.324504
Loss at epoch 900: 0.278793
Loss at epoch 1000: 0.246440
Loss at epoch 1100: 0.232806
Loss at epoch 1200: 0.221646
Loss at epoch 1300: 0.211485
Loss at epoch 1400: 0.202185
Loss at epoch 1500: 0.193616
Loss at epoch 1600: 0.185710
Loss at epoch 1700: 0.178448
Loss at epoch 1800: 0.171663
Loss at epoch 1900: 0.165354
Loss at epoch 2000: 0.159501


In [11]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.0011261  0.08132969 0.91754425]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [12]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.1970436e-06 1.2324920e-03 9.9876535e-01]
 [1.1260962e-03 8.1329688e-02 9.1754425e-01]
 [2.2168210e-07 1.6359952e-01 8.3640027e-01]
 [6.3965758e-06 8.4967339e-01 1.5032022e-01]
 [2.6112854e-01 7.2680873e-01 1.2062777e-02]
 [1.3780770e-01 8.6216480e-01 2.7532480e-05]
 [7.4204981e-01 2.5791308e-01 3.7120091e-05]
 [9.2180043e-01 7.8199029e-02 6.0299686e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [13]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.472190
Loss at epoch 500: 0.374904
Loss at epoch 1000: 0.229961
Loss at epoch 1500: 0.182154
Loss at epoch 2000: 0.150626
